In [ ]:
import requests
import pandas as pd
import numpy as np
import json
stop_api_url = "https://opendatakingston.cityofkingston.ca/api/records/1.0/search/?dataset=transit-gtfs-stops&q=&rows=799"
driveway_api_url = "https://opendatakingston.cityofkingston.ca/api/records/1.0/search/?dataset=driveways&q=&rows=-1&facet=material"
busroute_api_url = 'https://opendatakingston.cityofkingston.ca/api/records/1.0/search/?dataset=transit-gtfs-routes&q=&rows=-1&facet=route_short_name&exclude.route_long_name=999+Maintenance+-+Out+of+Service'
stop_response = requests.get(stop_api_url)
driveway_response = requests.get(driveway_api_url)
busroute_response = requests.get(busroute_api_url)

In [ ]:
driveway_response.status_code

200

In [ ]:
x = stop_response.json()
x_driveway = driveway_response.json()
x_busroute = busroute_response.json()

In [ ]:
transit_stops = pd.json_normalize(x,'records')
driveways = pd.json_normalize(x_driveway,'records')
busroute = pd.json_normalize(x_busroute,'records')

In [ ]:
driveways['fields.geo_point_2d']
transit_stops['fields.stop_coordinates']

0                      [44.25722, -76.57395]
1                      [44.26926, -76.55933]
2                      [44.26475, -76.54892]
3                      [44.26208, -76.54846]
4                      [44.25257, -76.54781]
                       ...                  
794                    [44.22496, -76.66337]
795                    [44.25365, -76.56864]
796    [44.2641910258681, -76.4985940698535]
797    [44.2656871117652, -76.5021851286293]
798     [44.259401941672, -76.5099725872279]
Name: fields.stop_coordinates, Length: 799, dtype: object

In [ ]:
import math

def haversine(coord1,coord2):
    # returns shortest distance between two points in metres
    
    # Radius of the Earth at Kingston. Assuming mean elevation of 93m above
    # sea level.
    R = 6367869
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi       = math.radians(lat2 - lat1)
    dlambda    = math.radians(lon2 - lon1)
    
    a = math.sin(dphi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    
    return 2*R*math.atan2(math.sqrt(a),math.sqrt(1-a))

In [ ]:
haversine(np.array(driveways['fields.geo_point_2d'][0]),np.array(transit_stops['fields.stop_coordinates'][0]))

14860.243499812255

The next section of code is trying to determine which routes service which stops. It is using map shading GPS coords of the bus routes to measure the distance of the route to the stop. A stop is said to be part of a bus route if the haversine formula measures a distance of \<100m of the shaded route to the stop.

In [ ]:
# The calculation needs to be complete for all bus stops. Stops need to be recorded in the dataframe. TBD.
stop_test = np.array(transit_stops['fields.stop_coordinates'][0])

for route, ind in enumerate(busroute['fields.shape.coordinates']):
    p = 0
    route_test = np.array(busroute['fields.shape.coordinates'][route][0])
    for ind in route_test:
        
        test_param = haversine(np.flip(route_test[p]),stop_test)
        if test_param<100:
            print(test_param)
            print(busroute['fields.route_short_name'][route])
            # Loop breaks if stop identified as being within 100m of bus route.
            break
            
        
        p = p+1


From here we create a matrix to document the shortest path from each 

In [ ]:
transit_lat = transit_stops['fields.stop_coordinates'].explode().to_numpy()[::2]
transit_long = transit_stops['fields.stop_coordinates'].explode().to_numpy()[1::2]
transit_coords = np.column_stack((transit_lat,transit_long))

driveway_lat = driveways['fields.geo_point_2d'].explode().to_numpy()[::2]
driveway_long =  driveways['fields.geo_point_2d'].explode().to_numpy()[1::2]

driveway_coords = np.column_stack((driveway_lat,driveway_long))

In [ ]:
stop_distance = np.ndarray(shape = [driveway_lat.size,transit_lat.size])
k = 0
l = 0

for i in driveway_coords:
    l = 0
    for j in transit_coords:
        stop_distance[k,l] = haversine(i,j)
        l = l+1
    k = k+1




In [ ]:
closest_stop = stop_distance.min(axis = 1)
print('median distance to stop: ',np.median(closest_stop),'m')
print(closest_stop.mean())

median distance to stop:  183.1881739352463 m
1211.0667639838714


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=63c92067-2a93-4cc0-850b-51f44255f10b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>